
### Librerías

In [1]:
import sys
import pandas as pd
import GetOldTweets3 as got
import sys
import matplotlib.pyplot as plt
from googletrans import Translator
import socket as sk
from datetime import *
import tweepy
import re
import string
from textblob import TextBlob
import nltk
from wordcloud import WordCloud

### Configuración de parámetros 

In [2]:
pd.options.display.max_columns = 5000
pd.options.display.max_rows = 5000
sk.setdefaulttimeout(600)

#Variable de autentificación en Twitter
OAUTH_TOKEN='1204477616398503936-7Z5WSIa0490S7HbIoPrDzosdA8aJKb'
OAUTH_SECRET='OCHWxZGQlrnaoNx4corTWHNZPrtGtItteYnow7nMDhtON'
CONSUMER_KEY='IRuah0diiQDx7ayuHaFmCOvTq'
CONSUMER_SECRET='FkYZE9vZXWMgWvovRjcqQaixPleh6ADXdD3XU05ZsIkscSFzXd'
auth = tweepy.OAuthHandler(CONSUMER_KEY,CONSUMER_SECRET)
auth.set_access_token(OAUTH_TOKEN,OAUTH_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, timeout=3600)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns',None)
pd.set_option('display.width',None)
pd.set_option('max_colwidth',-1)

### Ejecución de clases y métodos del proyecto

In [3]:
%run Metodos_Propios.ipynb
%run Gestion_Ficheros.ipynb
%run GetOldTweets.ipynb
%run NLP.ipynb
%run Extract_Twitter_v2.ipynb

# Método Principal - TFM Data Science

In [30]:
if __name__ == '__main__':
    
    #Control horario programa
    fecha_inicio = datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
    print('Comienzo del programa: ',fecha_inicio)
    
    # Definición de variables
    fichero = 'CSVs/info_tweets_el_corte_ingles_202003.csv'
    pattern = 'CSVs/info_tweets_el_corte_ingles*.csv'
    fichero_user = 'CSVs/following_info_el_corte_ingles.csv'
    fichero_tweets = 'CdM/Tweets_Info.csv'
    fichero_fact = 'CdM/Tweets_Fact_Info.csv'
    fichero_covid = 'CdM/Tweets_Covid_Info.csv'

    following_ids = []  
    calendario = calendar()
    df_tweets, df_tweets_final, df_following, df_fact_following = creacion_DataFrame()
    
    #Flujo de la información
    online = 0
    all_tweets = 0
    online_user = 0
    
    #Definición POO
    file = File_Manage()
    gotTweets = GoT(calendario)
    nlp = NLP()
    google = Translator()
    
    #Acceso a Twitter
    api = acceso_Twitter() 

    
    #Obtención por GetOldTweets
    if online == 1:
        print('Opción Online')
        print(gotTweets.getCalendar())
        df_tweets = gotTweets.get_Tweets(gotTweets.getCalendar())
    #Lectura de ficheros para ir más rápido ubicados en CSVs
    elif all_tweets == 1:
        file.setPattern(pattern)
        print('Opción Offline Tweets: ', file.getPattern())
        df_tweets = file.read_all_Tweets(file.getPattern()) 

    else: 
        file.setFile(fichero)
        print('Opción Un fichero: ', file.getFile())
        df_tweets = file.read_Tweets(file.getFile())
        
    df_tweets['valor'] = -10
    df_tweets['fecha'] = df_tweets['date'].astype(str).str[:10]
    df_tweets['fecha'] = df_tweets['fecha'].str.replace('-','/')
    df_tweets['objetividad'] = 0
    df_tweets['subjetividad'] = 0


    df_tweets['Tweet_punct'] = df_tweets['text'].str.lower().apply(lambda text: nlp.eliminar_signos(text))
    df_tweets['Tweet_traduccion'] = df_tweets['Tweet_punct'].apply(lambda text: google.translate(text,dest='es').text)
    df_tweets['Tweet_token'] = df_tweets['Tweet_traduccion'].apply(lambda text: nlp.tokenization(text))
    df_tweets['Tweet_novacias'] = df_tweets['Tweet_token'].apply(lambda text: nlp.quitar_palabras_vacias(text))
    df_tweets['Tweet_novacias'] = df_tweets['Tweet_novacias'].apply(lambda text: listToString(text))

    
    #Pasando el algoritmo de NLP para obtener el análisis de sentimiento
    df_tweets_final = nlp.analisis(df_tweets)
    
    #Se eliminan las columnas auxiliares
    df_tweets_final.drop(['Tweet_punct','Tweet_traduccion','Tweet_token'], axis=1, inplace = True)
    
    file.setFile(fichero_tweets)
    #file.writeCSV(df_tweets_final,file.getFile())
    
    #Para el caso de los usuarios followings, se crea un minimodelo de datos
    if online_user == 1:
        following_ids = get_following_ids(api)
        following_limit=[following_ids[n:n+99] for n in range(0, len(following_ids), 100)]
        df_following = pd.DataFrame(get_info_following(api,following_limit))
        file.setFile(fichero_user)
        file.writeCSV(df_following,file.getFile())
    else:
        file.setFile(fichero_user)
        df_following = file.readCSV(df_following,file.getFile())
     
    df_tweets.columns = ['id_text','created_at_tweet','name_account','text','geo','valor','fecha','objetividad','subjetividad','Tweet_novacias']

    df_following.columns = ['id_account','name_account','screen_name_account','created_at_account','location_account','followers_account','friends_account','description_account']
    df_fact_following = pd.merge(df_tweets, df_following,on=['name_account'],how='left')
    df_fact_following = df_fact_following[df_fact_following['id_account'].notnull()]
        
    
    file.setFile(fichero_fact)    
    #file.writeCSV(df_fact_following,file.getFile())
    
    # Información - Covid-19
    df_tweets = df_tweets[df_tweets['Tweet_novacias'].str.contains("coronavi|covid", na=False)]
    file.setFile(fichero_covid)    
    #file.writeCSV(df_tweets,file.getFile())

    
    fecha_fin = datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
    print('Hora de Finalizacion: ',fecha_fin)

Comienzo del programa:  08/17/2020, 18:09:50
Opción Un fichero:  CSVs/info_tweets_el_corte_ingles_202003.csv
Hora de Finalizacion:  08/17/2020, 19:03:43


In [31]:
df_tweets.head(100)

,id_text,created_at_tweet,name_account,text,geo,valor,fecha,objetividad,subjetividad,Tweet_novacias
7,1244764359156551682,2020-03-30 23:12:04+00:00,TierrayaguaVeng,CORONAVIRUS La doble cara de grandes empresas: donaciones millonarias a la sanidad pero menos contribución fiscal que en 2007 INDITEX y EL CORTE INGLÉS,NaN,0,2020/03/30,0.000000,0.000000,coronavirus doble cara grandes empresas donaciones millonarias sanidad menos contribución fiscal inditex corte inglés
53,1244701433552949248,2020-03-30 19:02:01+00:00,maria96MCR,Con todo esto del estado de alarma y #COVID19 nos hemos olvidado de lo verdaderamente importante... Es o no es ya primavera en el corte inglés???? O va para septiembre también?,NaN,0,2020/03/30,0.000000,0.000000,alarma covid olvidado verdaderamente importante primavera corte inglés va septiembre
58,1244693491382390785,2020-03-30 18:30:28+00:00,RamnSenra,"Mañana, a las 12.00h, Pablo e Irene se estarán haciendo pruebas del Covid en el Corte Inglés de Castellana...",NaN,0,2020/03/30,0.000000,0.000000,mañana h pablo irene haciendo pruebas covid corte inglés castellana
63,1244690717978628098,2020-03-30 18:19:27+00:00,Navarrainforma,Coronavirus: El BAN y el Corte Inglés de Pamplona donan alimentos a las Hermanitas de los pobres,NaN,0,2020/03/30,0.000000,0.000000,coronavirus ban corte inglés pamplona donan alimentos hermanitas pobres
135,1244629282015633411,2020-03-30 14:15:19+00:00,MBFestudio,Horario de los supermercados a día de hoy por la crisis del coronaviru... http://ladyblues.over-blog.es/2020/03/horario-de-los-supermercados-a-dia-de-hoy-por-la-crisis-del-coronavirus.html #coronavirus #mercadona #dia #carrefour #el_corte_ingles #Alcampo #Aldi #Ahorra_Mas #Lidl #supermercados #horarios,NaN,0,2020/03/30,0.000000,0.000000,horario supermercados día hoy crisis coronaviru httpladybluesoverblogeshorariodelossupermercadosadiadehoyporlacrisisdelcoronavirushtml coronavirus mercadona dia carrefour elcorteingles alcampo aldi ahorramas lidl supermercados horarios
169,1244593947516768256,2020-03-30 11:54:55+00:00,DiarioArriba,EL CORONAVIRUS CONTAGIA A EL CORTE INGLÉS: ERTE A 25.900 EMPLEADOS https://diarioarriba.es/2020/03/29/el-coronavirus-contagia-a-el-corte-ingles-erte-a-25-900-empleados/,NaN,0,2020/03/30,0.000000,0.000000,coronavirus contagia corte inglés erte empleados httpsdiarioarribaeselcoronaviruscontagiaaelcorteingleserteaempleados
255,1244425055767015424,2020-03-30 00:43:48+00:00,EmilioRodrigMen,"""Agradecimientos a Amancio Ortega y el Corte Ingles por sus donaciones que no las han hecho de palabra sino con hechos para ayudar a esta situación"" #españa #abogadosmadrid #madrid #abogacia #derechopenal #corrupción #coronavirus #sanidad",NaN,0,2020/03/30,0.000000,0.000000,agradecimientos amancio ortega corte ingles donaciones hecho palabra sino hechos ayudar situación españa abogadosmadrid madrid abogacia derechopenal corrupción coronavirus sanidad
266,1244393647241641989,2020-03-29 22:38:59+00:00,racle7,"Estos son los horarios de Mercadona, El Corte Inglés, Carrefour, Lidl, Dia y otros supermercados durante la crisis del coronavirus https://www.diariosur.es/malaga-capital/horarios-mercadona-corte-20200323135612-nt.html#vca=fixed-btn&amp;vso=rrss&amp;vmc=tw&amp;vli=M&#xE1;laga-Capital vía @diariosur",NaN,0,2020/03/29,0.000000,0.000000,horarios mercadona corte inglés carrefour lidl dia supermercados crisis coronavirus httpswwwdiariosuresmalagacapitalhorariosmercadonacortenthtmlvcafixedbtnampvsorrssampvmctwampvlimxelagacapital vía diariosur
268,1244392815158874115,2020-03-29 22:35:41+00:00,MiguelBPedro,#COVID19 Esta é a realidade pura e dura. As ofertas de máscaras e luvas da Inditex e el corte inglês não são mais que campanhas de marketing encobertas e gratuitas em todos os meios. Mas pagar impostos cada vez menos. Artigo de hoje de @_CarrascoLara no @_infoLibre,NaN,0,2020/03/29,0.000000,0.000000,covid pura dura realidad ofertas mascarillas guantes inditex corte ingles campañas marketing encubiertas gratuitas medios